# Using Writer's MCP package

This notebook shows you how to connect AI agents to Writer's ecosystem of tools using the [**Writer MCP (Model Context Protocol) SDK**](https://dev.writer.com/home/mcp-server) package. After completing this guide, you can chain together multi-step workflows without hardcoding API calls.

This example demonstrates the integration using LangChain, but the MCP package works with other frameworks as well.

---

## Prerequisites

Before starting, ensure you have the following:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)
- Basic familiarity with Python and [LangChain](https://dev.writer.com/home/integrations/langchain)

## Installation and setup

Install the Writer SDK MCP package globally using npm. This provides the MCP server that enables interaction with Writer's AI tools and services.


In [ ]:
!npm install -g writer-sdk-mcp

Set the `WRITER_API_KEY` environment variable. It is recommended to set this value in a `.env` file at the root of the project; however, this tutorial sets it directly as an environment variable if a `.env` file is not present.

In [ ]:
import getpass
import os

os.environ["WRITER_API_KEY"] = getpass.getpass()

## Starting the MCP server

Start the Writer MCP server on port 3000. This server acts as a bridge between your Python code and Writer's services, providing access to tools for content generation, Knowledge Graph operations, and file processing. The server runs in the background using subprocess.


In [ ]:
import subprocess

# Start MCP server on port 3000 (can change if needed)
server = subprocess.Popen(
    ["npx", "-y", "writer-sdk-mcp", "--transport=http", "--port=3000"],
    env=os.environ,
)
print("✅ Writer MCP Server running at http://localhost:3000")


## Example: building an assistant with LangChain

This example creates an assistant using the [**Writer LangChain SDK integration**](https://dev.writer.com/home/integrations/langchain), which connects the **Writer MCP SDK** (Node package) to the **LangChain framework**.

This setup allows your LangChain agents to call Writer's MCP tools directly through the [**LangChain MCP integration**](https://docs.langchain.com/oss/python/langchain/mcp).

### Python dependencies

Install **LangChain** with the **Writer MCP integration**, which provides:

- **[`langchain`](https://docs.langchain.com/oss/python/langchain/overview)**: the AI agent framework for building and orchestrating agents
- **[`langchain-mcp-adapters`](https://docs.langchain.com/oss/python/langchain/mcp)**: integration to connect LangChain agents to MCP tools
- **[`langchain-writer`](https://dev.writer.com/home/integrations/langchain)**: access Writer models like Palmyra X5 from LangChain agents


In [ ]:
%pip install -U langchain

In [ ]:
%pip install langchain-mcp-adapters

## System prompt configuration

Define the system prompt that guides the LangChain Agent's behavior. This prompt instructs the agent to:

- Act autonomously in reading, writing, and analyzing information
- Use Writer's MCP-connected tools for file processing, editing, and Knowledge Graph interactions
- Provide clear, human-readable explanations and summaries of its work


In [ ]:
system_prompt = """
You are a research and writing assistant built on the Writer SDK and connected to the Model Context Protocol (MCP).

Your goals:
1. Help users research technical, creative, or analytical topics.
2. Produce clear, accurate, and well-structured text.
3. When using tools, explain what you're doing and why, unless instructed otherwise.
4. Be concise and factual in tone.

Always prioritize correctness, clarity, and tool-assisted reasoning.
"""

## Creating the agent

The following code creates a `LangChainWriterAgent` class that:

- Connects to a local MCP server running on port 3000
- Uses Writer's Palmyra X5 model for natural language processing and reasoning
- Connects the model to the [`MultiServerMCPClient`](https://docs.langchain.com/oss/python/langchain/mcp), enabling access to available tools
- Provides an interactive chat interface for research and development tasks


In [ ]:
import asyncio
import subprocess
import os
import time
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langchain_writer import ChatWriter

# Make sure your WRITER_API_KEY is set
os.environ["WRITER_API_KEY"] = os.getenv("WRITER_API_KEY", "YOUR_API_KEY_HERE")

class LangChainWriterAgent:
    """Lightweight research agent using Writer's MCP tools via LangChain."""

    def __init__(self, system_prompt: str, server, mcp_url="http://localhost:3000"):

        # Connect to MCP
        self.client = MultiServerMCPClient(
            {
                "research": {
                    "transport": "streamable_http",
                    "url": mcp_url,
                }
            }
        )
        self.model = ChatWriter(model="palmyra-x5")
        self.system_prompt = system_prompt
        self.agent = None
        self.server = server

    async def init_agent(self):
        """Initialize LangChain agent with tools from MCP."""
        tools = await self.client.get_tools()
        self.agent = create_agent(self.model, tools, system_prompt=self.system_prompt)

    async def chat(self, query: str) -> str:
        """Send a query to the agent and stream results."""
        if not self.agent:
            await self.init_agent()

        final_response = ""
        async for chunk in self.agent.astream(
            {"messages": [{"role": "user", "content": query}]},
            stream_mode="updates"
        ):
            for step, data in chunk.items():
                text = data['messages'][-1].content_blocks[0]['text']
                print(text)
                final_response += text
        return final_response

    def stop(self):
        """Stop the MCP server."""
        self.server.terminate()
        self.server.wait()
        print("🛑 Writer MCP Server stopped.")


### Using the agent

Initialize the `LangChainWriterAgent` to connect to the Writer MCP server. The agent uses Palmyra X5 to process queries and access available MCP tools.

**Try this query:**

> "Tell me about no-code apps—what they are, how they work (general description, do not share names of no-code agents), and which model versions I can use."


In [ ]:
import nest_asyncio
import asyncio

os.environ["LANGSMITH_TRACING_V2"] = "false"

nest_asyncio.apply()

async def main():
    system_prompt = """
You are a research and writing assistant built on the Writer SDK and connected to the Model Context Protocol (MCP).

Your goals:
1. Help users research technical, creative, or analytical topics.
2. Produce clear, accurate, and well-structured text.
3. When using tools, explain what you're doing and why.
4. Be concise and factual in tone.
"""
    agent = LangChainWriterAgent(system_prompt=system_prompt, server=server)
    try:
        query = "Tell me about no-code apps—what they are, how they work (general, do not share names of no-code agents), and which model versions I can use."
        response = await agent.chat(query)
        print("\n\n✅ Final Response:\n", response)
    finally:
        agent.stop()

asyncio.run(main())
